In [336]:
from fastkml import  kml
import pandas as pd
import re
from bs4 import BeautifulSoup
import numpy as np

In [244]:
file = "./Data/Tourist Attractions/TOURISM.kml"

with open(file, 'rt', encoding="utf-8") as myfile:
    doc = myfile.read()

soup = BeautifulSoup(doc, 'lxml-xml')
soup    

<?xml version="1.0" encoding="utf-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/kml/2.2 http://schemas.opengis.net/kml/2.2.0/ogckml22.xsd http://www.google.com/kml/ext/2.2 http://code.google.com/apis/kml/schema/kml22gx.xsd">
<Document id="TOURISM">
<name>TOURISM</name>
<Snippet/>
<Folder id="FeatureLayer0">
<name>TOURISM</name>
<Snippet/>
<Placemark id="ID_00000">
<name>www.yoursingapore.com/en/see-do-singapore/culture-heritage/heritage-discovery/chinatown-heritage-centre.html</name>
<Snippet/>
<description>&lt;html xmlns:fo="http://www.w3.org/1999/XSL/Format" xmlns:msxsl="urn:schemas-microsoft-com:xslt"&gt;

&lt;head&gt;

&lt;META http-equiv="Content-Type" content="text/html"&gt;

&lt;meta http-equiv="content-type" content="text/html; charset=UTF-8"&gt;

&lt;/head&gt;

&lt;body style="margin:0px 0px 0px 0px;overflow:auto;background:#FFFFFF;"&

In [242]:
description_list = soup.find_all('description')
len(description_list)

106

# Extract all attractions details

In [337]:
attr_dict = {}
des_cnt = 0

for description in description_list:
    html_soup = BeautifulSoup(description.text, 'lxml')
    tables = html_soup.find_all('table')

    description_cnt = 0
    rows = tables[0].find_all('tr')       
    for r in range(len(rows)):
        cols = rows[r].find_all('td')
        if len(cols) == 2:
            attribute = cols[0].text.replace('<td>', '').replace('</td>', '')
            content = cols[1].text.replace('<td>', '').replace('</td>', '')
            if attribute not in attr_dict:
                attr_dict[attribute] = [content]
            else:
                attr_dict[attribute].append(content)

    des_cnt += 1 

# check all same number
for k, v in attr_dict.items():
    print(k, len(v))

SHAPE 106
URL Path 106
PHOTOURL 106
Image Text 106
Image By 106
NAME 106
Last Modified 106
Latitude 106
Longtitude 106
ADDRESSSTREETNAME 106
ADDRESSPOSTALCODE 106
DESCRIPTION 106
HYPERLINK 106
Description 106
Opening Hours 106
INC_CRC 106
FMEL_UPD_D 106
X_ADDR 106
Y_ADDR 106


In [355]:
pd.set_option('display.max_colwidth', 20)
df = pd.DataFrame(attr_dict)
df.head(3)

,SHAPE,URL Path,PHOTOURL,Image Text,Image By,NAME,Last Modified,Latitude,Longtitude,ADDRESSSTREETNAME,ADDRESSPOSTALCODE,DESCRIPTION,HYPERLINK,Description,Opening Hours,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR
0,Point,www.yoursingapor...,www.yoursingapor...,Learn more about...,Joel Chua DY,Chinatown Herita...,2015-11-02T10:16...,1.28351,103.84435,48 Pagoda Street,<Null>,Experience how S...,http://www.singa...,At the Chinatown...,"Daily, 9am – 8pm...",255C03B58EE7B003,1/9/2017 2:59:32 PM,29227.7061,29549.5371
1,Point,www.yoursingapor...,www.yoursingapor...,The Thian Hock K...,Joel Chua DY,Thian Hock Keng ...,2015-11-02T10:20...,1.28094,103.84763,158 Telok Ayer S...,<Null>,Beautifully rest...,http://www.thian...,Thian Hock Keng ...,"Daily, 7.30am – ...",D5D3E3930E0C117F,1/9/2017 2:59:32 PM,29592.7445,29265.3615
2,Point,www.yoursingapor...,www.yoursingapor...,Discover the his...,Joel Chua DY,Eurasian Heritag...,2015-11-02T10:13...,1.31007,103.89942,139 Ceylon Road,<Null>,This engaging he...,http://www.euras...,This engaging he...,Tuesday to Sunda...,9CB05BD01BA1715C,1/9/2017 2:59:32 PM,35356.4676,32486.4992


### Extract Coordinates

In [366]:
coord_list = soup.find_all('coordinates')
print(len(coord_list))

latitude = np.empty
longtitude = np.empty
for coord in coord_list:
        long_lat = coord.text.split(',')
        latitude = np.append(latitude, long_lat[1])
        longtitude = np.append(longtitude, long_lat[0])       

df['latitude'] = latitude[1:]
df['longtitude'] = longtitude[1:]

df.head(3)

106


,SHAPE,URL Path,PHOTOURL,Image Text,Image By,NAME,Last Modified,Latitude,Longtitude,ADDRESSSTREETNAME,...,Opening Hours,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,category,sub_category,description,latitude,longtitude
0,Point,www.yoursingapor...,www.yoursingapor...,Learn more about...,Joel Chua DY,Chinatown Herita...,2015-11-02T10:16...,1.28351,103.84435,48 Pagoda Street,...,"Daily, 9am – 8pm...",255C03B58EE7B003,1/9/2017 2:59:32 PM,29227.7061,29549.5371,culture heritage,heritage discovery,Experience how S...,1.283510000442852,103.8443500000813
1,Point,www.yoursingapor...,www.yoursingapor...,The Thian Hock K...,Joel Chua DY,Thian Hock Keng ...,2015-11-02T10:20...,1.28094,103.84763,158 Telok Ayer S...,...,"Daily, 7.30am – ...",D5D3E3930E0C117F,1/9/2017 2:59:32 PM,29592.7445,29265.3615,culture heritage,places of worship,Beautifully rest...,1.280939999723204,103.847629999838
2,Point,www.yoursingapor...,www.yoursingapor...,Discover the his...,Joel Chua DY,Eurasian Heritag...,2015-11-02T10:13...,1.31007,103.89942,139 Ceylon Road,...,Tuesday to Sunda...,9CB05BD01BA1715C,1/9/2017 2:59:32 PM,35356.4676,32486.4992,culture heritage,heritage discovery,This engaging he...,1.310070000376672,103.8994200001141


### Extract Category & Sub-category

In [367]:

category = np.empty
sub_category = np.empty

for url in df['URL Path']:
        elements = url.split('/')
        category = np.append(category, elements[-3].replace('-', ' '))
        sub_category = np.append(sub_category, elements[-2].replace('-', ' '))

df['category'] = category[1:]
df['sub_category'] = sub_category[1:]

df.head(2)


,SHAPE,URL Path,PHOTOURL,Image Text,Image By,NAME,Last Modified,Latitude,Longtitude,ADDRESSSTREETNAME,...,Opening Hours,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,category,sub_category,description,latitude,longtitude
0,Point,www.yoursingapor...,www.yoursingapor...,Learn more about...,Joel Chua DY,Chinatown Herita...,2015-11-02T10:16...,1.28351,103.84435,48 Pagoda Street,...,"Daily, 9am – 8pm...",255C03B58EE7B003,1/9/2017 2:59:32 PM,29227.7061,29549.5371,culture heritage,heritage discovery,Experience how S...,1.283510000442852,103.8443500000813
1,Point,www.yoursingapor...,www.yoursingapor...,The Thian Hock K...,Joel Chua DY,Thian Hock Keng ...,2015-11-02T10:20...,1.28094,103.84763,158 Telok Ayer S...,...,"Daily, 7.30am – ...",D5D3E3930E0C117F,1/9/2017 2:59:32 PM,29592.7445,29265.3615,culture heritage,places of worship,Beautifully rest...,1.280939999723204,103.847629999838


In [368]:
df["description"] = df[['DESCRIPTION', 'Description']].agg(' '.join, axis=1)
df.head(3)

,SHAPE,URL Path,PHOTOURL,Image Text,Image By,NAME,Last Modified,Latitude,Longtitude,ADDRESSSTREETNAME,...,Opening Hours,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,category,sub_category,description,latitude,longtitude
0,Point,www.yoursingapor...,www.yoursingapor...,Learn more about...,Joel Chua DY,Chinatown Herita...,2015-11-02T10:16...,1.28351,103.84435,48 Pagoda Street,...,"Daily, 9am – 8pm...",255C03B58EE7B003,1/9/2017 2:59:32 PM,29227.7061,29549.5371,culture heritage,heritage discovery,Experience how S...,1.283510000442852,103.8443500000813
1,Point,www.yoursingapor...,www.yoursingapor...,The Thian Hock K...,Joel Chua DY,Thian Hock Keng ...,2015-11-02T10:20...,1.28094,103.84763,158 Telok Ayer S...,...,"Daily, 7.30am – ...",D5D3E3930E0C117F,1/9/2017 2:59:32 PM,29592.7445,29265.3615,culture heritage,places of worship,Beautifully rest...,1.280939999723204,103.847629999838
2,Point,www.yoursingapor...,www.yoursingapor...,Discover the his...,Joel Chua DY,Eurasian Heritag...,2015-11-02T10:13...,1.31007,103.89942,139 Ceylon Road,...,Tuesday to Sunda...,9CB05BD01BA1715C,1/9/2017 2:59:32 PM,35356.4676,32486.4992,culture heritage,heritage discovery,This engaging he...,1.310070000376672,103.8994200001141


# Tidying up DataFrame

In [375]:
# Replace null values
df2.replace('<Null>', np.nan, inplace = True)
df2['ADDRESSPOSTALCODE'].replace('0', np.nan, inplace = True)

df2.head()

,NAME,ADDRESSSTREETNAME,ADDRESSPOSTALCODE,HYPERLINK,Opening Hours,X_ADDR,Y_ADDR,category,sub_category,description,latitude,longtitude
0,Chinatown Herita...,48 Pagoda Street,NaN,http://www.singa...,"Daily, 9am – 8pm...",29227.7061,29549.5371,culture heritage,heritage discovery,Experience how S...,1.283510000442852,103.8443500000813
1,Thian Hock Keng ...,158 Telok Ayer S...,NaN,http://www.thian...,"Daily, 7.30am – ...",29592.7445,29265.3615,culture heritage,places of worship,Beautifully rest...,1.280939999723204,103.847629999838
2,Eurasian Heritag...,139 Ceylon Road,NaN,http://www.euras...,Tuesday to Sunda...,35356.4676,32486.4992,culture heritage,heritage discovery,This engaging he...,1.310070000376672,103.8994200001141
3,Shophouses,NaN,NaN,NaN,NaN,28447.0623,28853.8642,architecture,historical,With many beauti...,1.277218600264318,103.8373355996684
4,Pinnacle@Duxton,1 Cantonment Road,NaN,http://www.pinna...,50th storey Skyb...,28901.624,28662.7257,architecture,modern,The soaring line...,1.275490000070019,103.8414200003462


In [385]:
final = pd.DataFrame(data = df2[["NAME", "category", "sub_category", "latitude", "longtitude", "description", 
                    "ADDRESSSTREETNAME", 'X_ADDR', 'Y_ADDR', 'HYPERLINK']])

final.columns = ['name', 'category', 'sub_category', 'latitude', 'longtitude', 'description', 
                'address', 'x_addr', 'y_addr', 'url']

final

,name,category,sub_category,latitude,longtitude,description,address,x_addr,y_addr,url
0,Chinatown Herita...,culture heritage,heritage discovery,1.283510000442852,103.8443500000813,Experience how S...,48 Pagoda Street,29227.7061,29549.5371,http://www.singa...
1,Thian Hock Keng ...,culture heritage,places of worship,1.280939999723204,103.847629999838,Beautifully rest...,158 Telok Ayer S...,29592.7445,29265.3615,http://www.thian...
2,Eurasian Heritag...,culture heritage,heritage discovery,1.310070000376672,103.8994200001141,This engaging he...,139 Ceylon Road,35356.4676,32486.4992,http://www.euras...
3,Shophouses,architecture,historical,1.277218600264318,103.8373355996684,With many beauti...,NaN,28447.0623,28853.8642,NaN
4,Pinnacle@Duxton,architecture,modern,1.275490000070019,103.8414200003462,The soaring line...,1 Cantonment Road,28901.624,28662.7257,http://www.pinna...
...,...,...,...,...,...,...,...,...,...,...
101,"Kusu Island, Sin...",nature wildlife,singapore islands,1.2230964997547,103.860523199782,If you’re lookin...,Kusu Island,31027.7192,22869.3386,http://www.sento...
102,Jurong Bird Park...,nature wildlife,fun with animals,1.318706000436972,103.7064420000953,Jurong Bird Park...,2 Jurong Hill,13879.8679,33441.6866,http://www.birdp...
103,HortPark,nature wildlife,parks gardens,1.279069000226187,103.7997239999188,Lose yourself in...,33 Hyderabad Road,24261.1939,29058.496,http://www.npark...
104,Adventure Cove W...,recreation leisure,fun things to do,1.257338799985209,103.8195254000482,Make a big splas...,8 Sentosa Gatewa...,26464.9175,26655.6626,http://www.rwsen...


# Convert to CSV

In [387]:
final.to_csv('./Data/Tourist Attractions/Tourist Attractions List.csv', index=False)